DiffDIS
====

**High-Precision Dichotomous Image Segmentation via Probing Diffusion Capacity**

 * Paper: https://arxiv.org/abs/2410.10105

![DiffDIS Overview](../assets/diffdis_overview.png)



! Installation

```bash
git clone https://github.com/qianyu-dlut/DiffDIS.git
cd DiffDIS

pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118
pip install -r requirements.txt
pip install -e diffusers-0.30.2/
```



 * Download pre-trained model (sd-ultra)
```python
...: from huggingface_hub import snapshot_download
...: 
...: # Replace this with your target directory
...: local_dir = "./models/sd-turbo"
...: 
...: # Download entire repository snapshot
...: snapshot_download(
...:     repo_id="stabilityai/sd-turbo",
...:     local_dir=local_dir,
...:     local_dir_use_symlinks=False  # Set to False to copy files instead of symlinks
...: )
```

 * Download the checkpoint from [Google Drive](https://drive.google.com/drive/folders/1NKmUbn9BiV7xYy_1c2khIBAuOQNuYAdR)

```bash
pip install gdown

# Right click on the file and get the link

gdown https://drive.google.com/uc?id=FILE_ID
```

In [2]:
import os
import sys

import numpy as np
import torch
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from torchvision import transforms
from transformers import CLIPTextModel, CLIPTokenizer

sys.path.append("DiffDIS-repo")
from core.diffdis_pipeline import DiffDISPipeline
from diffusers import (
    DDPMScheduler,
    UNet2DConditionModel_diffdis,
    AutoencoderKL,
)

from utils.seed_all import seed_all 
from utils.utils import check_mkdir
from utils.config import diste1,diste2,diste3,diste4,disvd
from utils.image_util import resize_res

to_pil = transforms.ToPILImage()


In [3]:
use_seperate = False
    
checkpoint_path = "DiffDIS-repo/checkpoints/"
pretrained_model_path = "DiffDIS-repo/models/sd-turbo"
      
# Diffusion denoising steps, more steps results in higher accuracy but slower inference speed
denoise_steps = 1
# Number of predictions to be ensembled, more inference gives better results but runs slower.
ensemble_size = 1
# Run with half-precision (16-bit float), might lead to suboptimal result.
half_precision = False
# Maximum resolution of processing. 0 for using input image resolution. Default: 1024.
processing_res = 1024
# Match input resolution for output depth. Default: False.
match_input_res = True
# Random seed for reproducibility
seed = None
# Inference batch size. Default: 0 (will be set automatically).
batch_size = 1

match_output_input_res = False

    
# -------------------- Preparation --------------------
# Random seed
if seed is None:
    import time
    seed = int(time.time())
seed_all(seed)

# -------------------- Device --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import ttach as tta
transforms = tta.Compose(
[
    tta.HorizontalFlip(),
    tta.Scale(scales=[0.75, 1, 1.25], interpolation='bilinear', align_corners=False),
])
    
dtype = torch.float16 if half_precision else torch.float32

In [4]:
# -------------------- Model --------------------
    
vae = AutoencoderKL.from_pretrained(
    pretrained_model_path, subfolder='vae'
)
scheduler = DDPMScheduler.from_pretrained(
    pretrained_model_path, subfolder='scheduler'
)
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_path, subfolder='text_encoder'
)
tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_path, subfolder='tokenizer'
)
unet = UNet2DConditionModel_diffdis.from_pretrained(
    checkpoint_path, subfolder="unet",
    in_channels=8, sample_size=96,
    low_cpu_mem_usage=False,
    ignore_mismatched_sizes=False,
    class_embed_type='projection',
    projection_class_embeddings_input_dim=4,
    mid_extra_cross=True,
    mode = 'DBIA', use_swci = True
)

pipe = DiffDISPipeline(
    unet=unet,
    vae=vae,
    scheduler=scheduler,
    text_encoder=text_encoder,
    tokenizer=tokenizer
)
print("Using Seperated Modules")
    

pipe = pipe.to(device);

Using Seperated Modules
